In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [35]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    
    # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    try:
        # get the login button
        login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/li[@title="Login"]')))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # complete logiin info
# =====================================================================================

    # get the username tab  
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,"usr")))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pas")))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")

    # get the login tab to enter the username and password
    try:
        # login_tab_css = "submit"
        login_tab = driver.find_element(By.ID,  "lgn")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
        for i in range(3):
            driver.back()
            i+=1
    except Exception as e:
        print(f"No login tab found and the error is {e}")
    
    
    results = [] # to append the dict values later on
    run = 0
    while True:

        details = {}
        
        # basic car details
        try:
            det_card_main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auc_info right"]')))
            if det_card_main:
                
                # car lot
                try:
                    lot =  WebDriverWait(det_card_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//span[@id="span_0"])[1]'))).text.strip()
                    if lot:
                        details["Lot"] = lot
                    else:
                        details["Lot"] = "na"

                except Exception as e:
                    print("No lot number")
                
                # car name
                try:
                    title =  WebDriverWait(det_card_main, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@class="lot-name"]'))).text.strip()
                    if title:
                        details["Title"] = title
                    else:
                        details["Title"] = "na"

                except Exception as e:
                    print("No title")
                    
                # other data .//div[@class="custom_field_title"]
                try:
                    info_card =  WebDriverWait(det_card_main, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@id="custom_field_section"]')))
                    if info_card:
                        info_rows = WebDriverWait(info_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="custom_field_sub_section"]')))
                        for info_row in info_rows:
                            info_lbl = info_row.find_element(By.XPATH, './/div[@class="custom_field_title"]').text.strip()
                            info_val = info_row.find_element(By.XPATH, './/div[@class="custom_field_value"]').text.strip()
                            if info_lbl and info_val:
                                details[info_lbl] = info_val
                except Exception as e:
                    print("No other information here ")
        except Exception as e:
            print("no car main card")
            # break
        
        # images
        try:
            imgs_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="mediaThumbnails"]')))
            if imgs_card:
                imgs = WebDriverWait(imgs_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@data-zoom-id="Zoom-1"]')))
                imgs_lst = [img.get_attribute("href") for img in imgs]
                details['Images'] = ", ".join(imgs_lst)
        except Exception as e:
            print(f"No images found {e}")
        
        results.append(details)  
        try:
            next_button = driver.find_element(By.XPATH, './/a[text()="Next"]')
            if next_button:
                next_button.click()
                # break
                # time.sleep()
        except Exception as e:
            print("No next button found hence no more cars are available")
            break
        
        # run+=1
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("data2.csv", index=False)
    
    time.sleep(5)
    driver.quit()
path = "https://auction.tsaauctions.co.uk/lot-details/index/catalog/1825/lot/822614/Peugeot-207-Sport-1360cc-3-Door?url=%2Fauctions%2Fcatalog%2Fid%2F1825"
scrape(path)

No images found Message: 
Stacktrace:
	GetHandleVerifier [0x00EF0283+25139]
	(No symbol) [0x00E7B234]
	(No symbol) [0x00D504A3]
	(No symbol) [0x00D98338]
	(No symbol) [0x00D9868B]
	(No symbol) [0x00DE0F62]
	(No symbol) [0x00DBCF84]
	(No symbol) [0x00DDE6E1]
	(No symbol) [0x00DBCD36]
	(No symbol) [0x00D8BD29]
	(No symbol) [0x00D8D064]
	GetHandleVerifier [0x011FB143+3215603]
	GetHandleVerifier [0x012122BA+3310186]
	GetHandleVerifier [0x0120C4D2+3286146]
	GetHandleVerifier [0x00F89C80+654384]
	(No symbol) [0x00E845BD]
	(No symbol) [0x00E814A8]
	(No symbol) [0x00E81647]
	(No symbol) [0x00E73D20]
	BaseThreadInitThunk [0x76DBFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776B82AE+286]
	RtlGetAppContainerNamedObjectPath [0x776B827E+238]

No images found Message: 
Stacktrace:
	GetHandleVerifier [0x00EF0283+25139]
	(No symbol) [0x00E7B234]
	(No symbol) [0x00D504A3]
	(No symbol) [0x00D98338]
	(No symbol) [0x00D9868B]
	(No symbol) [0x00DE0F62]
	(No symbol) [0x00DBCF84]
	(No symbol) [0x00DDE6E1]
	

In [36]:
df = pd.read_csv("data2.csv")
df

,Lot,Title,BUYER'S PREMIUM,VAT STATUS,REGISTRATION NO,DATE FIRST REGISTERED (D.F.R),MILES/KMS,DECLARATION,TRANSMISSION,FUEL TYPE,COLOUR,REGISTRATION DOC,MOT,MOT EXPIRY DATE,VAT,Images,T/LOSS,CATEGORY,REMARKS,TAX
0,LOT 1,Peugeot 207 Sport - 1360cc 3 Door,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,SW08KKA,20-Jun-08,"63,108",Correct,Manual,Petrol,Black,Yes,T/A,25-Mar-25,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,NaN,NaN
1,LOT 2,Subaru Legacy Ren - 1994cc Estate,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,BF56PUY,02-Feb-07,"151,000",Correct,Manual,Petrol,Grey,Yes,T/A,12-Feb-26,S,https://auction.thainstoneonline.co.uk/images/...,YES,N,History File in Car,NaN
2,LOT 3,Kia Sorento Xs - 2497cc Estate,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,SH09PXO,08-May-09,"116,663",Correct,Manual,Diesel,Black,Yes,NaN,- -,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,NaN,NaN
3,LOT 4,Fiat Panda Active Eco - 1108cc 5 Door,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,SW09GXP,17-Jul-09,"50,490",Correct,Manual,Petrol,Silver,Yes,NaN,- -,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,NaN,NaN
4,LOT 5,Skoda Roomster 3 16v 105 - 1598cc 5 Door,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,DG57LHT,24-Sep-07,"95,964",Correct,Manual,Petrol,Blue,Yes,Yes,13-Nov-25,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,9 Service Stamps,NaN
5,LOT 6,Mitsubishi Asx 5 Di-d 4x4 Auto - 2268cc 5 Door,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,GK17CYG,29-Mar-17,"130,139",Correct,Auto,Diesel,Blue,T/A,T/A,30-Jun-25,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,NaN,NaN
6,LOT 7,Audi A3 Sport Tdi - 1598cc Convertible,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,EN11WSO,07-Jul-11,"136,102",Correct,Manual,Diesel,White,Yes,NaN,- -,S,https://auction.thainstoneonline.co.uk/images/...,YES,N,Trade-In,NaN
7,LOT 8,-,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,0,0,0,NaN,NaN,0,0,0,0,0,0,https://auction.thainstoneonline.co.uk/images/...,0,0,0,0.0
8,LOT 9,Suzuki Vitara Sz-t Ddis - 1598cc 5 Door,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,SC16YHN,21-Jun-16,"83,656",Correct,Manual,Diesel,Grey,Yes,T/A,04-Dec-25,S,https://auction.thainstoneonline.co.uk/images/...,NaN,NaN,Long MOT,NaN
9,LOT 10,-,A Combined Buyers/Online Premium of 5% + Vat @...,NO VAT,0,0,0,NaN,NaN,0,0,0,0,0,0,https://auction.thainstoneonline.co.uk/images/...,0,0,0,0.0


# image downloader

In [37]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[['REGISTRATION NO', 'Images']]


def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["REGISTRATION NO"] # separate reg nums
        image_urls = row["Images"]
        
        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\SW08KKA\SW08KKA_1_1047742_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_2_1047743_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_3_1047744_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_4_1047745_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_5_1047746_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_6_1047747_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_7_1047748_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_8_1047749_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_9_1047750_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_10_1047751_xl.jpg
Downloaded: Images\SW08KKA\SW08KKA_11_1047752_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_1_1048788_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_2_1048789_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_3_1048790_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_4_1048791_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_5_1048792_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_6_1048793_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_7_1048794_xl.jpg
Downloaded: Images\BF56PUY\BF56PUY_8_1048795_xl.jpg
Downloaded

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'float'